In [7]:
!python train.py --name test_mvp_switch --model pix2pix --n_epochs 1 --niter 1

🚀 启动训练脚本...
   加载扫描参数: ./datasets/simulation/resolution_distorsion/resolution_distorsion_simu_scan.hdf5
   加载数据文件: ./datasets/simulation/resolution_distorsion/resolution_distorsion_simu_dataset_iq.hdf5
   IQ数据形状: (75, 128, 382)
   发射角度数: 75
   中心频率: 5.21 MHz
   采样频率: 5.21 MHz
   声速: 1540.0 m/s
   时间零点扩展到所有角度
   验证数据完整性...
   ✅ PICMUS数据加载完成并验证通过
----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                    epoch: latest                        
              epoch_count: 1                             
                 gan_mode: vanilla                       
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 1   

In [ ]:
!python train.py --name test_mvp_clean2 --model pix2pix --n_epochs 1 --niter 1 --niter_decay 0

🚀 启动训练脚本...
   加载扫描参数: ./datasets/simulation/resolution_distorsion/resolution_distorsion_simu_scan.hdf5
   加载数据文件: ./datasets/simulation/resolution_distorsion/resolution_distorsion_simu_dataset_iq.hdf5
   IQ数据形状: (75, 128, 382)
   发射角度数: 75
   中心频率: 5.21 MHz
   采样频率: 5.21 MHz
   声速: 1540.0 m/s
   时间零点扩展到所有角度
   验证数据完整性...
   ✅ PICMUS数据加载完成并验证通过
----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                    epoch: latest                        
              epoch_count: 1                             
                 gan_mode: vanilla                       
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 1   

In [ ]:
# 测试n_layers判别器（确保这个选项还能工作）
!python train.py --name test_nlayers --model pix2pix --netD n_layers --n_layers_D 4 --n_epochs 1 --niter 1

🚀 启动训练脚本...
   加载扫描参数: ./datasets/simulation/resolution_distorsion/resolution_distorsion_simu_scan.hdf5
   加载数据文件: ./datasets/simulation/resolution_distorsion/resolution_distorsion_simu_dataset_iq.hdf5
   IQ数据形状: (75, 128, 382)
   发射角度数: 75
   中心频率: 5.21 MHz
   采样频率: 5.21 MHz
   声速: 1540.0 m/s
   时间零点扩展到所有角度
   验证数据完整性...
   ✅ PICMUS数据加载完成并验证通过
----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                    epoch: latest                        
              epoch_count: 1                             
                 gan_mode: vanilla                       
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 1   

In [ ]:
!python -c "from data_process import load_dataset; from options.train_options import TrainOptions; opt = TrainOptions().parse(); dataset = load_dataset(opt, 'train', 0); print(f'数据集大小: {len(dataset)} 对图像')"

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                    epoch: latest                        
              epoch_count: 1                             
                 gan_mode: vanilla                       
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 1                             
                  isTrain: True                          	[default: None]
                lambda_L1: 1                             
                     load: ./img_data                    
                load_iter: 0                             	[default: 0]
                       lr: 0.0002                        
 

In [ ]:
# check_data.py
import os
import h5py
import numpy as np
from cubdl.PlaneWaveData import PICMUSData
from cubdl.example_picmus_torch import load_datasets, create_network, mk_img
b
print("="*50)
print("检查AUGAN训练数据")
print("="*50)

# 1. 检查文件是否存在
files_needed = [
    "./datasets/simulation/resolution_distorsion/resolution_distorsion_simu_dataset_iq.hdf5",
    "./datasets/simulation/resolution_distorsion/resolution_distorsion_simu_scan.hdf5"
]

for f in files_needed:
    if os.path.exists(f):
        print(f"✅ 找到: {f}")
        # 检查文件大小
        size_mb = os.path.getsize(f) / (1024*1024)
        print(f"   大小: {size_mb:.1f} MB")
    else:
        print(f"❌ 缺失: {f}")

# 2. 尝试加载数据
try:
    print("\n测试数据加载...")
    plane_wave_data = load_datasets("simulation", "resolution_distorsion", "iq")
    print(f"✅ 数据加载成功")
    print(f"   IQ数据形状: {plane_wave_data.idata.shape}")
    print(f"   角度数量: {len(plane_wave_data.angles)}")
    print(f"   阵元数量: {plane_wave_data.ele_pos.shape[0]}")
    
    # 3. 测试DAS重建
    print("\n测试DAS重建...")
    das, iqdata, xlims, zlims = create_network(plane_wave_data, [1])
    single_img = mk_img(das, iqdata)
    print(f"✅ 单角度图像: {single_img.shape}")
    
    das_multi, _, _, _ = create_network(plane_wave_data, list(range(75)))
    multi_img = mk_img(das_multi, iqdata)
    print(f"✅ 多角度图像: {multi_img.shape}")
    
    print("\n📊 数据统计:")
    print(f"   可生成训练对数: 根据代码，应该是1对图像")
    print(f"   图像尺寸: {single_img.shape}")
    print(f"   数据类型: {single_img.dtype}")
    
except Exception as e:
    print(f"❌ 错误: {e}")
    
print("\n" + "="*50)
print("结论:")
if 'single_img' in locals() and 'multi_img' in locals():
    print("✅ 数据足够进行训练！")
    print("   - 有输入图像A（单角度）")
    print("   - 有目标图像B（多角度）")
    print("   - 可以开始训练")
else:
    print("❌ 数据不足，需要检查PICMUS数据集")

In [3]:
!python -c "from data_process import load_dataset; from options.train_options import TrainOptions; opt = TrainOptions().parse(); dataset = load_dataset(opt, 'train', 0); print(f'数据集大小: {len(dataset)}')"

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                    epoch: latest                        
              epoch_count: 1                             
                 gan_mode: vanilla                       
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 1                             
                  isTrain: True                          	[default: None]
                lambda_L1: 1                             
                     load: ./img_data                    
                load_iter: 0                             	[default: 0]
                       lr: 0.0002                        
 